<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
# use requests.get() method with the provided static_url
# assign the response to a object

import requests

# The provided static URL

# Perform HTTP GET request
response = requests.get(static_url)

# Check if the request was successful
print("Response Status Code:", response.status_code)

# Assign HTML content to a variable
html_content = response.text

Response Status Code: 200


Create a `BeautifulSoup` object from the HTML `response`


In [6]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
import requests
from bs4 import BeautifulSoup

# Get HTML page
response = requests.get(static_url)

# Check if request was successful
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    print("Page title:", soup.title.string)
else:
    print("Failed to load page. Status code:", response.status_code)

Page title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [7]:
# Use soup.title attribute
# URL of the static Falcon 9 HTML page


# Perform HTTP GET request
response = requests.get(static_url)

# Create BeautifulSoup object
soup = BeautifulSoup(response.text, 'html.parser')

#  Print the page title
print("Page Title:", soup.title.string)

Page Title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [8]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`

soup = BeautifulSoup(response.text, 'html.parser')

# Step 3: Find all tables in the page
tables = soup.find_all('table')
print(" Number of tables found:", len(tables))

# Step 4: Select the first table (or adjust index if needed)
table = tables[0]  # Assume the Falcon 9 table is the first one

# Step 5: Helper function to clean header names
def extract_column_from_header(row):
    """
    Clean and extract a column name from a header cell (<th>).
    """
    if (row.br):
        row.br.extract()  # Remove <br> tags
    if row.a:
        row.a.extract()   # Remove <a> links
    if row.sup:
        row.sup.extract() # Remove <sup> tags
    
    # Get the text content of the header
    colunm_name = row.get_text(" ", strip=True)

    # Filter out headers that are purely digits or empty
    if colunm_name and not colunm_name.strip().isdigit():
        return colunm_name
    return None

# Step 6: Extract all header cells from the selected table
headers = table.find_all('th')

# Step 7: Clean and store column names
html_tables = [extract_column_from_header(th) for th in headers]
html_tables = [name for name in html_tables if name is not None]  # Remove None values

#  Step 8: Print the list of cleaned column names
print(" Column Names Extracted:")
print(html_tables)

 Number of tables found: 25
 Column Names Extracted:
[]


Starting from the third table is our target table contains the actual launch records.


In [11]:
tables = soup.find_all('table')
print("Number of tables found:", len(tables))

# Step 2: Detect the Falcon 9 table automatically
falcon9_table = None
for idx, tbl in enumerate(tables):
    headers = tbl.find_all('th')
    header_texts = [th.get_text(strip=True) for th in headers]
    print(f"\nTable {idx} headers preview:", header_texts[:5])  # show first 5 header texts
    # Check if this table has known Falcon 9 header keywords
    if any("Date" in h or "Outcome" in h for h in header_texts):
        falcon9_table = tbl
        print(f"\n✅ Falcon 9 table likely found at index {idx}")
        break

if falcon9_table is None:
    raise ValueError("Falcon 9 table not found on the page!")

# Step 3: Clean headers
def extract_column_from_header(row):
    """
    Clean and extract a column name from a header cell (<th>).
    """
    if row.br:
        row.br.extract()  # Remove <br> tags
    if row.a:
        row.a.extract()   # Remove <a> links
    if row.sup:
        row.sup.extract() # Remove <sup> tags
    colunm_name = row.get_text(" ", strip=True)
    # Filter out empty headers or purely numeric ones
    if colunm_name and not colunm_name.strip().isdigit():
        return colunm_name
    return None

# Step 4: Extract headers
headers = falcon9_table.find_all('th')
html_tables = [extract_column_from_header(th) for th in headers]
html_tables = [name for name in html_tables if name is not None]  # Remove None values

print("\n✅ Cleaned Column Names Extracted:")
print(html_tables)

# Optional: If you want to view the first few rows of this table
rows = falcon9_table.find_all('tr')
for i, row in enumerate(rows[:3]):  # show first 3 rows
    print("\nRow", i, ":", row.text.strip()[:200])  # limit text to 200 chars

Number of tables found: 25

Table 0 headers preview: []

Table 1 headers preview: []

Table 2 headers preview: ['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]']

✅ Falcon 9 table likely found at index 2

✅ Cleaned Column Names Extracted:
['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']

Row 0 : Flight No.

Date andtime ()

 

Launch site

Payload

Payload mass

Orbit

Customer

Launchoutcome

Row 1 : 1

4 June 2010,18:45

F9 v1.0[7]B0003.1[8]

CCAFS,SLC-40

Dragon Spacecraft Qualification Unit



LEO

SpaceX

Success

Failure[9][10](parachute)

Row 2 : First flight of Falcon 9 v1.0.[11] Used a boilerplate version of Dragon capsule which was not designed to separate from the second stage.(more details below) Attempted to recover the first stage by pa


You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [15]:
# Find all tables on the page
tables = soup.find_all('table')
print("Number of tables found:", len(tables))

# Automatically detect the Falcon 9 launch table
first_launch_table = None
for idx, tbl in enumerate(tables):
    headers = tbl.find_all('th')
    header_texts = [th.get_text(strip=True) for th in headers]
    print(f"Table {idx}: headers preview ->", header_texts[:5])  # preview first 5 headers
    # If table has expected Falcon 9 headers
    if any("Date" in h or "Outcome" in h for h in header_texts):
        first_launch_table = tbl
        print(f"\n Falcon 9 launch table found at index {idx}")
        break

# Make sure a table was found
if first_launch_table is None:
    raise ValueError("Falcon 9 table not found. Check the table indices.")

# Now extract the column names safely
column_names = []

# Apply find_all('th') on the selected table
header_cells = first_launch_table.find_all('th')

for th in header_cells:
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

# Display the cleaned column names
print("\n Cleaned Column Names:")
print(column_names)

Number of tables found: 25
Table 0: headers preview -> []
Table 1: headers preview -> []
Table 2: headers preview -> ['Flight No.', 'Date andtime ()', '', 'Launch site', 'Payload']

 Falcon 9 launch table found at index 2

 Cleaned Column Names:
['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


Check the extracted column names


In [16]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [17]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [19]:
extracted_row = 0
launch_data = []  # List to hold all the dictionaries (each row)

# Extract each table
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # Get table rows
    for rows in table.find_all("tr"):
        # Check if first table heading is a number (corresponding to launch number)
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
            else:
                flag = False
        else:
            flag = False

        # Get all <td> cells in the row
        row = rows.find_all('td')

        # If flight number is valid, process the row
        if flag:
            extracted_row += 1

            # Initialize dictionary for this row
            launch_dict = {}

            # --- Extract and save values ---
            # Flight Number
            launch_dict['Flight No.'] = int(flight_number)

            # Date and Time
            datatimelist = date_time(row[0])  # custom function
            launch_dict['Date'] = datatimelist[0].strip(',')
            launch_dict['Time'] = datatimelist[1]

            # Booster Version
            bv = booster_version(row[1])  # custom function
            if not bv:
                bv = row[1].a.string  # fallback
            launch_dict['Version Booster'] = bv

            # Launch Site
            launch_dict['Launch Site'] = row[2].a.string

            # Payload
            payload = row[3].a.string if row[3].a else row[3].text.strip()
            launch_dict['Payload'] = payload

            # Payload Mass
            payload_mass = get_mass(row[4])  # custom function
            launch_dict['Payload Mass (kg)'] = payload_mass

            # Orbit
            launch_dict['Orbit'] = row[5].a.string

            # Customer
            launch_dict['Customer'] = row[6].a.string if row[6].a else row[6].text.strip()

            # Launch Outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch Outcome'] = launch_outcome.strip()

            # Booster Landing
            booster_landing = landing_status(row[8])  # custom function
            launch_dict['Booster Landing'] = booster_landing

            # Append this row's dictionary to the list
            launch_data.append(launch_dict)

# ✅ Print number of rows extracted
print(f"\nTotal rows extracted: {extracted_row}")

# ✅ (Optional) Convert to Pandas DataFrame
import pandas as pd
launch_df = pd.DataFrame(launch_data)
print("\n✅ First few rows of DataFrame:")
print(launch_df.head())


Total rows extracted: 121

✅ First few rows of DataFrame:
   Flight No.             Date   Time   Version Booster Launch Site  \
0           1      4 June 2010  18:45  F9 v1.07B0003.18       CCAFS   
1           2  8 December 2010  15:43  F9 v1.07B0004.18       CCAFS   
2           3      22 May 2012  07:44  F9 v1.07B0005.18       CCAFS   
3           4   8 October 2012  00:35  F9 v1.07B0006.18       CCAFS   
4           5     1 March 2013  15:10  F9 v1.07B0007.18       CCAFS   

                                Payload Payload Mass (kg) Orbit Customer  \
0  Dragon Spacecraft Qualification Unit                 0   LEO   SpaceX   
1                                Dragon                 0   LEO     NASA   
2                                Dragon            525 kg   LEO     NASA   
3                          SpaceX CRS-1          4,700 kg   LEO     NASA   
4                          SpaceX CRS-2          4,877 kg   LEO     NASA   

  Launch Outcome Booster Landing  
0        Success      

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [20]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
